## TN-Models Workbook 

We will be playing around with tn-models and going through some examples in this sample workbook this library is built with some predefined defaults that exist integrate with django by default, the api can be extended for other frameworks as well. 


## Simplest implementation 

In this snippet we will be creating a very simple api client using just the defaults associated with the tn-models library by passing in the minimum required variables 


**client**: an api client e.g axios that is already configured


**baseUrl**: this should just be the uri of the route you would like to use, it is appended to the baseurl of the client:


    | in theory you could also pass in a full url here instead of configuring it in the client 


**models**: an object containing shapes of your date types, these will be used for inputs and outputs


    - ***entity***: this is the only requiredkey of the models 
 



In [1]:
/**
 * 
 * run this for some common utilities to be available in the environemnt 
 * use let if you intend to reassign the variable
 * 
 */
import z from 'npm:zod@3.22.4'
import {createApi, } from 'npm:@thinknimble/tn-models@3.1.0'
import axios from 'npm:axios@0.21.1'


let baseUrl = "http://localhost:8000"
let res




In [3]:




const client = axios.create({
    baseURL: baseUrl,
})

let userShape = {
    id: z.string(),
    email: z.string().email(),
    firstName: z.string(),
    lastName: z.string(),

}

let usersApi = createApi({
    client,
    baseUri: '/api/users/',
    models: {
        entity: userShape,
    }
})

console.log('list: \n',await usersApi.list())
console.log('retrieve: \n',await usersApi.retrieve('1eec99ef-541e-474b-b54b-8c4688982bb5'))



list: 
 { count: 1000, next: null, previous: null, results: [] }


Error: Request failed with status code 404

## Extending the capabilities while still using the defaults 

The above example is a basic implementation of an api we have only passed in the minimum required fields with this we have get and delete method access to our api with the pre-defined list & retrieve endpoints. 

Building off our basic example we will also enable access to the predefined create, update and delete endpoints, extending our get only api to work with post, patch and delete. To do this we only need to provide one more key to our models 

**models**: ...

    - ***create***: we can reuse the same shape as the entity or create a new one

In [4]:

let userCreateShape = {
    email: z.string().email(),
    firstName: z.string(),
    lastName: z.string(),

}

usersApi = createApi({
    client,
    baseUri: '/api/users/',
    models: {
        entity: userShape,
        create: userCreateShape,
    }
})



console.log("create: \n", await usersApi.create({email: "some1@example.com", firstName:"some", lastName:"example"}))
// not implemented in mock server yet
usersApi.update("id", {email: ""})





create: 
 {
  id: "520eb9b9-9638-4185-951c-41f8f8acdcc3",
  email: "some1@example.com",
  firstName: "some",
  lastName: "example"
}


Promise {
  <rejected> TypeError: Cannot use 'in' operator to search for 'id' in id
    at file:///Users/paribaker/Library/Caches/deno/npm/registry.npmjs.org/@thinknimble/tn-models/2.5.0/dist/index.js:584:16
    at Generator.next (<anonymous>)
    at file:///Users/paribaker/Library/Caches/deno/npm/registry.npmjs.org/@thinknimble/tn-models/2.5.0/dist/index.js:65:61
    at new Promise (<anonymous>)
    at __async (file:///Users/paribaker/Library/Caches/deno/npm/registry.npmjs.org/@thinknimble/tn-models/2.5.0/dist/index.js:49:10)
    at updateBase (file:///Users/paribaker/Library/Caches/deno/npm/registry.npmjs.org/@thinknimble/tn-models/2.5.0/dist/index.js:579:30)
    at file:///Users/paribaker/Library/Caches/deno/npm/registry.npmjs.org/@thinknimble/tn-models/2.5.0/dist/index.js:606:12
    at Generator.next (<anonymous>)
    at file:///Users/paribaker/Library/Caches/deno/npm/registry.npmjs.org/@thinknimble/tn-models/2.5.0/dist/index.js:65:61
    at new Promise (<anonymous>)
}

## Additional options with defaults 

Continuing just with our defaults we can also add filtering & pagination to our api 

First we defined extraFilters as part of our models(we can do this with an inline shape or by decalring it elsewhere)

**As of now the fitlers can only be of string type** that means we have to convert different options to string like date, true, false, null, etc.

Here are some examples of variations to these that we typically encounter

- some_field__isnull > someField__isnull='True'
- some_date > someDate="2024-12-2310:00:00.0000T"




In [5]:
usersApi = createApi({
    client,
    baseUri: '/api/users/',
    models: {
        entity: userShape,
        create: userShape,
        extraFilters:{
            search: z.string(),
        }
    }
})
// not implemented in server yet
await usersApi.list({filters:{search: "john"}, pagination:{page:1}}).then(console.log).catch(console.error)



{
  count: 1000,
  next: null,
  previous: null,
  results: [
    {
      id: "520eb9b9-9638-4185-951c-41f8f8acdcc3",
      email: "some1@example.com",
      firstName: "some",
      lastName: "example"
    }
  ]
}


We can also add additional methods to our api that we can call with `customCalls` or with the short hand `csc`

In [ ]:
const customCreate = createCustomServiceCall({
    inputShape: createUserShape,
    outputShape: userShape,
  
    cb: async ({ client, input, utils }) => {
      const clean = utils.toApi(input)
      const res = await client.post('/api/users/', clean)

      return utils.fromApi(res.data)
    },
  })

usersApi = createApi({
    client,
    baseUri: '/api/users/',
    models: {
        entity: userShape,
        create: userShape,
        extraFilters:{
            search: z.string(),
        }
    },
    customCalls:{
      customCreate
    }
})
// not implemented in server yet
await usersApi.csc.customCreate({}).then(console.log).catch(console.error)

